<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice Using APIs

_Instructor: Aymeric Flaisler_

---

In this lab we will practice using some popular APIs to retrieve and store data.

In [1]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline

## Exercise 1: get data from Sheetsu

---

[Sheetsu](https://sheetsu.com/) is an online service that allows you to access any Google spreadsheet from an API. This can be a very powerful way to share a dataset with colleagues as well as to create a mini centralized data storage that is simpler to edit than a database.

A Google Spreadsheet with Wine data can be found [here](https://docs.google.com/spreadsheets/d/1mZ3Otr5AV4v8WwvLOAvWf3VLxDa-IeJ1AVTEuavJqeo/).

It can be accessed through sheetsu API at this endpoint: https://sheetsu.com/apis/v1.0/2899e32f4844

**Questions:**

1. Use the requests library to access the document. Inspect the response text. What kind of data is it?
- Check the status code of the response object. What code is it?
- Use the appropriate libraries and read functions to read the response into a Pandas Dataframe.
- Once you've imported the data into a dataframe, check the value of the 5th line: what's the price?

In [2]:
# You can either post or get info from this API
api_base_url = 'https://sheetsu.com/apis/v1.0su/4ce9c3459aad'

In [3]:
# A:
data=requests.get(api_base_url)

In [4]:
data.text

'{"error":"Too many rows."}'

In [5]:
data.status_code

402

## Exercise 2: post data to Sheetsu

---

We've learned how to read data, but it'd be great if we could also write data. For this we will need to use a _POST_ request.

1. Use the post command to add the `post_data` to the spreadsheet.
- What status did you get? How can you check that you actually added the data correctly?
- In this exercise, your classmates are adding data to the same spreadsheet. What happens because of this? Is it a problem? How could you mitigate it?

In [6]:
post_data = {
'Grape' : ''
, 'Name' : 'My wonderful wine'
, 'Color' : 'R'
, 'Country' : 'US'
, 'Region' : 'Sonoma'
, 'Vinyard' : ''
, 'Score' : '10'
, 'Consumed In' : '2015'
, 'Vintage' : '1973'
, 'Price' : '200'
}

In [7]:
# A:


## Exercise 3: IMDB TV Shows

---

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.

Here we will use a combination of scraping and API calls to find the ratings and networks of famous television shows.

### 3.A Get the top TV Shows

The Internet Movie Database contains data about movies and TV shows. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/toptv/?ref_=nv_tp_tv250_2 contains the list of the top 250 tv shows of all time. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

> **Hint:** movie_ids look like this: `tt2582802`
> _Everything after "/title/" and before "/?"_

In [65]:
# A:
data2=requests.get('http://www.imdb.com/chart/toptv/?ref_=nv_tp_tv250_2')


In [66]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(data2.text)

In [67]:
counter=0
lst=[]
for link in soup.find_all('a'):
    tt=re.findall(r'/title/(.*)/',str(link.get('href')))
    if tt:
        counter+=1
        lst.append(tt[0])
        #print(tt)
    #print(link.text)
counter=0
final_lst=[]
for i in lst:
    counter+=1
    if counter%2 != 0:
        final_lst.append(i)
print(len(final_lst))

251


In [68]:
#final_lst.pop(0)
#the first id is not included in the list of top 250 movies.

'tt0245429'

### 3.B Get data on the top movies

Although the Internet Movie Database does not have a public API, an open API exists at http://www.tvmaze.com/api.

Use this API to retrieve information about each of the 250 TV shows you have extracted in the previous step.
1. Check the documentation of tvmaze's api to learn how to request show data by id.
- Define a function that returns a python object with select information for a given id.
    - Show name
    - Rating (avg)
    - Genre(s)
    - Network name
    - Premiere date
    - Status
> Tip: the json object can easily be converted into a python dictionary.

- Store the gathered information in a Pandas Dataframe.

As the target information is in json format you will need `json.loads()` in order to gather it.

In [140]:
# A:
html=requests.get('http://api.tvmaze.com/lookup/shows?imdb=tt0944947')

In [145]:
import json
json.loads(html.text)

{'id': 82,
 'url': 'http://www.tvmaze.com/shows/82/game-of-thrones',
 'name': 'Game of Thrones',
 'type': 'Scripted',
 'language': 'English',
 'genres': ['Drama', 'Adventure', 'Fantasy'],
 'status': 'Running',
 'runtime': 60,
 'premiered': '2011-04-17',
 'officialSite': 'http://www.hbo.com/game-of-thrones',
 'schedule': {'time': '21:00', 'days': ['Sunday']},
 'rating': {'average': 9.3},
 'weight': 99,
 'network': {'id': 8,
  'name': 'HBO',
  'country': {'name': 'United States',
   'code': 'US',
   'timezone': 'America/New_York'}},
 'webChannel': {'id': 22,
  'name': 'HBO Go',
  'country': {'name': 'United States',
   'code': 'US',
   'timezone': 'America/New_York'}},
 'externals': {'tvrage': 24493, 'thetvdb': 121361, 'imdb': 'tt0944947'},
 'image': {'medium': 'http://static.tvmaze.com/uploads/images/medium_portrait/143/359013.jpg',
  'original': 'http://static.tvmaze.com/uploads/images/original_untouched/143/359013.jpg'},
 'summary': '<p>Based on the bestselling book series <i>A Song o

In [146]:
def api_to_df(final_lst):
    temp_dict={
        'name':[],
        'rating':[],
        'genre':[],
        'network_name':[],
        'premiered':[],
        'status':[],}
    for i in final_lst:
        html=requests.get('http://api.tvmaze.com/lookup/shows?imdb={}'.format(i))
        try:
            html=json.loads(html.text)
        except:
            print('ID:',i,' ','Error_type:',html,'ID not available in website')
            continue
        try:
            temp_dict['name'].append(html['name'])
        except:
            temp_dict['name'].append(None)
        try:
            temp_dict['rating'].append(html['rating']['average'])
        except:
            temp_dict['rating'].append(None)
        try:
            temp_dict['genre'].append(html['genres'])
        except:
            temp_dict['genre'].append(None)
        try:
            temp_dict['network_name'].append(html['network']['name'])
        except:
            temp_dict['network_name'].append(None)
        try:
            temp_dict['premiered'].append(html['premiered'])
        except:
            temp_dict['premiered'].append(None)
        try:
            temp_dict['status'].append(html['status'])
        except:
            temp_dict['status'].append(None)
    return pd.DataFrame(temp_dict)

In [147]:
df=api_to_df(final_lst)

ID: tt1508238   Error_type: <Response [404]> ID not available in website
ID: tt0096548   Error_type: <Response [404]> ID not available in website
ID: tt0121220   Error_type: <Response [404]> ID not available in website
ID: tt0090509   Error_type: <Response [404]> ID not available in website
ID: tt0108855   Error_type: <Response [404]> ID not available in website
ID: tt0088484   Error_type: <Response [404]> ID not available in website
ID: tt0092324   Error_type: <Response [404]> ID not available in website
ID: tt0314979   Error_type: <Response [404]> ID not available in website
ID: tt0203082   Error_type: <Response [404]> ID not available in website
ID: tt8595766   Error_type: <Response [404]> ID not available in website
ID: tt0088509   Error_type: <Response [404]> ID not available in website
ID: tt4093826   Error_type: <Response [404]> ID not available in website
ID: tt1230180   Error_type: <Response [404]> ID not available in website
ID: tt6205862   Error_type: <Response [404]> ID not

In [148]:
df

,name,rating,genre,network_name,premiered,status
0,Planet Earth II,9.6,[Nature],BBC One,2016-11-06,Ended
1,Band of Brothers,9.3,"[Drama, Action, War]",HBO,2001-09-09,Ended
2,Game of Thrones,9.3,"[Drama, Adventure, Fantasy]",HBO,2011-04-17,Running
3,Planet Earth,9.3,[Nature],BBC One,2006-03-05,Ended
4,Breaking Bad,9.3,"[Drama, Crime, Thriller]",AMC,2008-01-20,Ended
5,The Wire,9.3,"[Drama, Crime]",HBO,2002-06-02,Ended
6,Cosmos,9.3,[],FOX,2014-03-09,Running
7,Rick and Morty,9.4,"[Comedy, Adventure, Science-Fiction]",Adult Swim,2013-12-02,Running
8,Cosmos,10.0,[],PBS,1980-09-28,Ended
9,Blue Planet II,9.4,[Nature],BBC One,2017-10-29,Ended


In [149]:
len(df.dropna())

182